In [ ]:
## run this on pnyq..

import numpy as np
import time

#from scipy.interpolate import interp1d

mnist = np.load("mnist-original.npy", allow_pickle= True)

X = mnist.item().get("data").T / 255
y = mnist.item().get("label")[0]

weights = np.load('model.npy',allow_pickle=True)

fc1w = weights.item().get('fc1w')
fc1b = weights.item().get('fc1b')

fc2w = weights.item().get('fc2w')
fc2b = weights.item().get('fc2b')

fc3w = weights.item().get('fc3w')
fc3b = weights.item().get('fc3b')


'''
import torch
model = torch.load("MNIST.pth")
#model = np.load("model.npy",allow_pickle= True)
fc1w = np.array(model['fc1.weight'])
fc1b = np.array(model['fc1.bias'])

fc2w = np.array(model['fc2.weight'])
fc2b = np.array(model['fc2.bias'])

fc3w = np.array(model['fc3.weight'])
fc3b = np.array(model['fc3.bias'])
print(fc1w.shape, fc2w.shape, fc3w.shape)
print(fc2w.T.shape)
'''

'\nimport torch\nmodel = torch.load("MNIST.pth")\n#model = np.load("model.npy",allow_pickle= True)\nfc1w = np.array(model[\'fc1.weight\'])\nfc1b = np.array(model[\'fc1.bias\'])\n\nfc2w = np.array(model[\'fc2.weight\'])\nfc2b = np.array(model[\'fc2.bias\'])\n\nfc3w = np.array(model[\'fc3.weight\'])\nfc3b = np.array(model[\'fc3.bias\'])\nprint(fc1w.shape, fc2w.shape, fc3w.shape)\nprint(fc2w.T.shape)\n'

In [ ]:

batch_size = 64

In [ ]:


# assume hw can do only int8 multiplication

def sign_bit(a):
    if(a>0):
        return 1
    else:
        return -1

sign_bit_map = np.vectorize(sign_bit)

def feed_foward(X0):
    X1 = np.matmul(X0, fc1w.T) + np.tile(fc1b, (batch_size, 1))
    A1 = np.tanh(X1)
    
    A1_q = sign_bit_map(A1)
    fc2w_q = sign_bit_map(fc2w)
    
    HW_result = np.matmul(A1_q, fc2w_q.T)

    X2 = HW_result + np.tile(fc2b, (batch_size, 1))
    A2 = np.tanh(X2)

    X3 = np.matmul(A2, fc3w.T) + np.tile(fc3b, (batch_size, 1))
    return X3





In [ ]:
%%time
prediction = []
for idx in range(len(X)//batch_size):
    xs = X[batch_size * idx:batch_size * idx + batch_size]
    ys = y[batch_size * idx:batch_size * idx + batch_size]
    outputs = feed_foward(xs)
    for output, yk in zip(outputs, ys):
        prediction.append(np.argmax(output) == (yk))

score = np.mean(prediction) * 100

print(score)

89.03247941445562
CPU times: user 2min 1s, sys: 9.83 s, total: 2min 10s
Wall time: 3min 6s


In [ ]:
from pynq import allocate
from pynq import Overlay
import numpy as np
import struct
import ctypes

In [ ]:
custom_hw = Overlay("matmulaccel10.bit")

In [20]:
class matmul_overlay(Overlay):
    def __init__(self, bitfile_name, SIZE=64):
        super().__init__(bitfile_name)
        #self.saxpy_hw = self.krnl_saxpy2_0
        self.abuf = allocate((SIZE*SIZE*2, ) , dtype=np.float64)
        self.bbuf = allocate((SIZE*SIZE*2, ) , dtype=np.float64)
        self.obuf = allocate((SIZE*SIZE, ) , dtype=np.int8)
        #interrupt 생략
        
    
    def python_float_to_uint(self, num):
        return ctypes.c_uint.from_buffer(ctypes.c_float(num)).value
    
    def register_map(self):
        return self.register_map
    
    def ip_dict(self):
        return self.saxpy_hw.ip_dict()
    
    def run(self, x, y):

        np.copyto(self.abuf, x.flatten())
        np.copyto(self.bbuf, y.flatten())
        
        self.in1_dma.sendchannel.transfer(self.abuf)
        self.in2_dma.sendchannel.transfer(self.bbuf)
        self.out_dma.recvchannel.transfer(self.obuf)
        self.in1_dma.sendchannel.wait()
        self.in2_dma.sendchannel.wait()
        self.out_dma.recvchannel.wait()
        
        obuf_64by64 = np.reshape(self.obuf, (64,64))
        #obuf_64by64 = obuf_64by64.astype(np.float64)
        #obuf_64by64 = self.obuf.resize((64,64))
        
        return obuf_64by64
    
    def assert_by_sw(self, x, y):
        def sign_chk(a):
            if (a>=0):
                return 1
            else:
                return -1
        import time
        sign_chk_map = np.vectorize(sign_chk)
        x_sw = sign_chk_map(a)
        y_sw = sign_chk_map(b)
        start_sw = time.time()
        xy = np.matmul(x_sw,y_sw)
        end_sw = time.time()
        start_hw = time.time()
        xy_hw = self.run(x,y)
        end_hw = time.time()
        comparison = np.isclose(xy.flatten(),xy_hw.flatten())
        for i in comparison:
            assert(i == True)
        
        print(end_sw-start_sw, end_hw-start_hw)
        
        return True

In [21]:
matmul_hw = matmul_overlay("matmulaccel10.bit")


In [22]:
def feed_foward2(X0):
    X1 = np.matmul(X0, fc1w.T) + np.tile(fc1b, (batch_size, 1))
    A1 = np.tanh(X1)
    
    #matmul_hw = matmul_overlay("matmulaccel.bit")
    
    A1_q = A1.astype(np.float64)
    fc2w_q = fc2w.astype(np.float64)
    HW_result = matmul_hw.run(A1_q,fc2w_q.T)
    
    X2 = HW_result + np.tile(fc2b, (batch_size, 1))
    A2 = np.tanh(X2)
    #print(A2[0].dtype, X2[0].dtype)
    X3 = np.matmul(A2, fc3w.T) + np.tile(fc3b, (batch_size, 1))
    return X3

In [23]:
%%time
prediction2 = []
for idx in range(len(X)//batch_size):
    xs = X[batch_size * idx:batch_size * idx + batch_size]
    ys = y[batch_size * idx:batch_size * idx + batch_size]
    outputs2 = feed_foward2(xs)
    for output, yk in zip(outputs2, ys):
        prediction2.append(np.argmax(output) == (yk))

score = np.mean(prediction2) * 100

print(score)

89.03247941445562
CPU times: user 2min 46s, sys: 15 s, total: 3min 1s
Wall time: 5min 16s
